In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import DataFrame

In [2]:
def max_stock(data):
    L = []
    for i in set(data["stock"]):
        counter = list(data["stock"].values).count(i)
        L.append(counter)                                        ###L为各股票出现的次数，即交易天数
    stock_id = list(set(data["stock"]))
    stock_m = stock_id[L.index(max(L))]                           ###选出交易天数最全的一只股票
    return stock_m

In [3]:
def best_perf(data, stock_eg, T, N1, N2, K):                                 ###data为原始数据，T为所要比较的涨跌幅的日期，N1，N2为T之前和T之后的时间跨度，K为最大成交总额及涨跌幅对比数

    data['stock'] = data['stock'].astype(str)                       ###更改stock的数据类型为字符型

    start = int(stock_eg[stock_eg['date'] ==  T].index.values-N1-1)-stock_eg.index[0]                  ###寻找T前N+1天的索引位置
    middle = int(stock_eg[stock_eg['date'] == T].index.values)-1-stock_eg.index[0]             ###寻找T日前一日的索引位置
    end = middle+N2                                                 ###统计涨跌幅最后一日的索引位置
    slc = stock_eg.iloc[[start, middle, middle+1, end],:]           ###选出所需计算的股票信息
    
    init = data[data['date'] == slc['date'].values[0]]
    middle = data[data['date'] == slc['date'].values[1]]
    determ = data[data['date'] == slc['date'].values[2]]
    fina = data[data['date'] == slc['date'].values[3]]
    secl_sum = init.append(middle).sort_index()
    secl_sum = secl_sum.append(determ).sort_index()
    secl_sum = secl_sum.append(fina).sort_index()                   ####将所有在T,T-N1-1,T-1，T-1+N2时有数据的股票信息合成一张表
    
    stock_intersect = list(set(init['stock']).intersection(set(middle['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(determ['stock'])))
    stock_intersect = list(set(stock_intersect).intersection(set(fina['stock'])))        ###选出在上述四个时段都有数据的股票
    
    stock_f = pd.DataFrame(columns = ['date', 'stock', 'close', 'turnover_sum', 'change', 'change_sum', 'turnover_N1', 'turnover_N2', 'change_N'])  ###建立一个空的dataframe
    
    for k in stock_intersect:
        starter = secl_sum[secl_sum['stock'] == k]
        turn_o = starter['turnover_sum']
        change_s = starter['change_sum']
        differ_t = turn_o.values[1]-turn_o.values[0]                  ###计算T前N1天的成交金额总量
        differ_t2 = turn_o.values[3]-turn_o.values[1]                 ###计算T后N2天的成交金额总量
        differ_c = change_s.values[3]-change_s.values[1]              ###计算T后N2天的涨跌幅
        stock_a = starter[starter['date'] == T]
        stock_a.insert(6,'turnover_N1',[differ_t])
        stock_a.insert(7,'turnover_N2',[differ_t2])
        stock_a.insert(8,'change_N',[differ_c/N2])                    ###插入N2天的平均日涨跌幅
        stock_f = stock_f.append(stock_a)

    stock_f = stock_f.sort_index()
    stock_f = stock_f[['date','stock','close','turnover_N1','turnover_N2','change_N']]
    
    stock_Tmax = stock_f.sort_values(by = ['turnover_N1'], ascending = False).head(K)     ###取出前N1天成交金额最大的K个股票
    stock_Cmax = stock_f.sort_values(by = ['change_N'], ascending = False).head(K)         ###取出N2天平均日涨跌幅最大的K个股票
    stock_best = list(set(stock_Tmax['stock']).intersection(set(stock_Cmax['stock'])))   ###取出涨跌幅及成交金额都是最大K的股票     
    
    stock_bestinf = stock_Cmax[stock_Cmax['stock'] == stock_best[0]]
    for i in stock_best[1:]:
        infom = stock_Cmax[stock_Cmax['stock'] == i]
        stock_bestinf = stock_bestinf.append(infom)                                ###获取这几只股票的具体信息
        stock_bestinf = stock_bestinf.sort_index()
    return stock_bestinf  

In [4]:
def statisc(data, stock_eg, N1, N2, K):
    
    statis = pd.DataFrame(columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change', 'count'])

    for i in stock_eg['date'].values[N1+1:-N2+1]:
        try:
            stock_bestinf = best_perf(data, stock_eg, i, N1, N2, K)
            size = stock_bestinf.iloc[:,0].size                   ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
            avg_turnN1 = stock_bestinf['turnover_N1'].mean()      ###求出满足条件股票的前N1日成交金额平均值
            avg_turnN2 = stock_bestinf['turnover_N2'].mean()      ###求出满足条件股票的后N2日成交金额平均值
            avg_change = stock_bestinf['change_N'].mean()         ###求出满足条件股票的日收益率平均值
            new_data = {'date':stock_bestinf['date'].values[0], 'avg_turnN1':avg_turnN1, 'avg_turnN2':avg_turnN2, 'avg_change':avg_change, 'count':size}
            new_add = pd.DataFrame(new_data, columns = ['date', 'avg_turnN1', 'avg_turnN2', 'avg_change', 'count'],index=[0])     ###将列表合并
            statis = statis.append(new_add, ignore_index=True)
        except:
            continue

    return statis

In [5]:
data = pd.read_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017/2017.csv',encoding='gbk',engine='python')

In [6]:
stock_m = 1
#stock_m = max_stock(data)
stock_eg = data[data['stock'] == stock_m]

In [7]:
statis = statisc(data, stock_eg, 10, 20, 100)
statis

,date,avg_turnN1,avg_turnN2,avg_change,count
0,2017-01-18,8.604112e+09,1.945946e+10,0.013371,6
1,2017-01-19,7.827315e+09,1.651508e+10,0.013184,7
2,2017-01-20,8.341427e+09,1.624903e+10,0.014294,7
3,2017-01-23,9.146765e+09,2.021314e+10,0.012761,6
4,2017-01-24,9.699318e+09,2.202378e+10,0.014293,4
5,2017-01-25,7.529423e+09,1.836173e+10,0.014857,7
6,2017-01-26,6.821371e+09,1.793309e+10,0.014500,7
7,2017-02-03,6.177595e+09,1.550982e+10,0.010927,12
8,2017-02-06,5.742867e+09,1.551276e+10,0.011196,6
9,2017-02-07,5.301671e+09,1.917674e+10,0.012698,5


In [8]:
statis.to_csv('D:/Documents/Go Abroad/实习/成交金额/Python数据/2017/2017-10-20.csv', index=False)

In [8]:
stock_eg

,date,stock,close,volume,turnover,value,turnover_sum,change,change_sum
0,2017-01-03,1,9.16,45984049,4.205952e+08,1.340216e+08,4.205952e+08,NaN,NaN
1,2017-01-04,1,9.16,44932953,4.115034e+08,1.340216e+08,8.320986e+08,0.000000,0.000000
2,2017-01-05,1,9.17,34437291,3.157697e+08,1.341679e+08,1.147868e+09,0.001092,0.001092
3,2017-01-06,1,9.13,35815420,3.271764e+08,1.335827e+08,1.475045e+09,-0.004362,-0.003270
4,2017-01-09,1,9.15,36108157,3.299946e+08,1.547996e+08,1.805039e+09,0.002191,-0.001080
5,2017-01-10,1,9.15,24105395,2.205751e+08,1.547996e+08,2.025614e+09,0.000000,-0.001080
6,2017-01-11,1,9.14,30343089,2.775532e+08,1.546304e+08,2.303168e+09,-0.001093,-0.002173
7,2017-01-12,1,9.15,42800677,3.918694e+08,1.547996e+08,2.695037e+09,0.001094,-0.001079
8,2017-01-13,1,9.16,43430137,3.976019e+08,1.549688e+08,3.092639e+09,0.001093,0.000014
9,2017-01-16,1,9.14,68316586,6.230258e+08,1.546304e+08,3.715665e+09,-0.002183,-0.002169


In [16]:
N1 = 55
N2 = 15
K = 100

for i in stock_eg['date'].values[227:-15]:
    try:     
        stock_bestinf = best_perf(data, stock_eg, i, N1, N2, K)
        size = stock_bestinf.iloc[:,0].size                 ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
        avg_change = stock_bestinf['change_N'].mean()       ###求出满足条件股票的日收益率平均值
        new_data = {'date':stock_bestinf['date'].values[0], 'avg_change':avg_change, 'count':size}
        new_add = pd.DataFrame(new_data, columns = ['date', 'avg_change', 'count'], index=[0])     ###将列表合并
        statis = statis.append(new_add, ignore_index=True)
    except:
        continue

In [17]:
statis

,date,avg_change,count
0,2017-12-07,0.010528,3
1,2017-12-08,0.012087,11


In [13]:
stock_eg['date'].values[227:-15]

array(['2017-12-07', '2017-12-08'], dtype=object)

In [ ]:
N1 = 231
N2 = 10
K = 100

for i in stock_eg['date'].values[N1+1:-N2]:
    stock_bestinf = best_perf(data, stock_eg, i, N1, N2, K)
    size = stock_bestinf.iloc[:,0].size                 ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
    avg_change = stock_bestinf['change_N'].mean()       ###求出满足条件股票的日收益率平均值
    new_data = {'date':stock_bestinf['date'].values[0], 'avg_change':avg_change, 'count':size}
    new_add = pd.DataFrame(new_data, columns = ['date', 'avg_change', 'count'],index=[0])     ###将列表合并
    statis = statis.append(new_add, ignore_index=True)

In [8]:
stock_bestinf = best_perf(data, stock_eg, '2017-11-20', 30, 10, 100)

In [9]:
stock_bestinf

,date,stock,close,turnover_N,change_N
93380,2017-11-20,830,15.00,1.964543e+10,0.015973
139822,2017-11-20,2049,42.46,2.824993e+10,0.028531
241459,2017-11-20,2497,13.96,2.362274e+10,0.013923
605612,2017-11-20,300618,214.09,2.439109e+10,0.029921
644905,2017-11-20,603993,6.22,2.870980e+10,0.016623
665983,2017-11-20,600460,11.27,3.295912e+10,0.024358
686514,2017-11-20,603799,81.27,4.577075e+10,0.013235


In [11]:
size = stock_bestinf.iloc[:,0].size         ###既是前段时间成交金额总额排名靠前的，也是涨幅排名靠前的
size

7

In [16]:
avg_change = stock_bestinf['change_N'].mean()
avg_change

0.020366266519649835

In [19]:
new_data = {'date':stock_bestinf['date'].values[0], 'avg_change':avg_change, 'count':size}

In [22]:
new_add = pd.DataFrame(new_data, columns = ['date', 'avg_change', 'count'],index=[0])     ###将列表合并

In [23]:
new_add

,date,avg_change,count
0,2017-11-20,0.020366,7
